# Rogue Trader support tables

## Introduction

This file creates assets_df and index_df russian and eglish versions.

### Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Folders

In [2]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
# Save and load folders
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'


### Files

#### Table of gamefiles ids

In [3]:
# list of files in work_folder
list_of_files = []
for root, dirs, files in os.walk(work_folder):           
        for file in files:
            if file.endswith(".jbp"):            
                list_of_files.append(os.path.join(root, file))
print(len(list_of_files))

109114


In [4]:
def create_assets(contents_dict):
    file_info = {}
    file_info['AssetId'] = contents_dict['AssetId']
    file_info['File'] = file
    return file_info

assets_df = pd.DataFrame()
for file in list_of_files:
    with open(file, encoding='latin-1' ) as user_file:
        try:
            file_contents = user_file.read()
            contents_dict = json.loads(file_contents)        
            file_info = create_assets(contents_dict)
            df = pd.DataFrame.from_dict(file_info, orient = 'index').T
            assets_df = pd.concat([assets_df,df],axis = 0)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',file)
assets_df = assets_df.reset_index(drop = True)
display(assets_df.head())



AssetId  \
0  23068e0962f9449d9d8e3dea732a5958   
1  d48e03729a253814ebb26d21b4e6a0b3   
2  5ebfb51da6304029b71e89fd45619ed3   
3  bcf25b9f55e2465a910309fb5aab117c   
4  ddb5c3520fe94966acb57592484fd542   

                                                                                                File  
0                                  e:\Steam\RogueTrader\WorkFolder\Blueprints\CameraRecorderData.jbp  
1                                    e:\Steam\RogueTrader\WorkFolder\Blueprints\Player Character.jbp  
2               e:\Steam\RogueTrader\WorkFolder\Blueprints\Player CharacterFemale_VisualSettings.jbp  
3                     e:\Steam\RogueTrader\WorkFolder\Blueprints\Player Character_VisualSettings.jbp  
4  e:\Steam\RogueTrader\WorkFolder\Blueprints\Achievements\PentakillAchievement_TechnicalCounter.jbp

#### Table english localization

In [5]:
#Search strings in index_file
index_df = pd.read_json(index_file).reset_index()
#unfold dict in the column "strings"
text = pd.json_normalize(index_df['strings'])
index_df = pd.concat([index_df,text['Text']],axis =1).drop('strings',axis = 1).rename(columns = {'Text':'text'})
display(index_df.head())

index  \
0  000393b6-487b-4e13-a1af-4702957a0ee8   
1  0004ce39-6474-47b1-a65f-85365b83a5b1   
2  0004e905-8ef5-420e-accc-e8dc056caf7e   
3  00074f46-4e59-4dcd-b118-d535a7673eba   
4  00092c30-751e-4fc9-a549-a7a717893fa7   

                                                                                                            text  
0                                                                                   Sword of Faith — Area Attack  
1                                                                           The {0} archetype has been mastered!  
2                                                                                                    Plasma Ring  
3  {n}Cassia offers you a reticent smile.{/n} "I foresaw this answer within the golden sparks of your thoughts."  
4                                                                                                    Steady Hand

### Table of russian localization

In [6]:
#Search strings in index_file
index_df_ru = pd.read_json(index_file_ru).reset_index()
#unfold dict in the column "strings"
text = pd.json_normalize(index_df_ru['strings'])
index_df_ru = pd.concat([index_df_ru,text['Text']],axis =1).drop('strings',axis = 1).rename(columns = {'Text':'text'})
display(index_df_ru.head())

index  \
0  000393b6-487b-4e13-a1af-4702957a0ee8   
1  0004ce39-6474-47b1-a65f-85365b83a5b1   
2  0004e905-8ef5-420e-accc-e8dc056caf7e   
3  00074f46-4e59-4dcd-b118-d535a7673eba   
4  00092c30-751e-4fc9-a549-a7a717893fa7   

                                                                                                text  
0                                                                        Меч веры — атака по области  
1                                                                        Архетип {0} успешно изучен!  
2                                                                                      Плазма-кольцо  
3  {n}Кассия сдержанно улыбается.{/n} "В золотистых всполохах ваших мыслей я предвидела этот ответ".  
4                                                                                        Верная рука

In [7]:
assets_df.to_csv(save_folder + r'\assets_df.csv', index=False)
index_df.to_csv(save_folder + r'\index_df.csv', index=False)
index_df_ru.to_csv(save_folder + r'\index_df_ru.csv', index=False)